In [109]:
import os
import sys
import json
import openai
from dotenv import load_dotenv

import psycopg
from psycopg.rows import dict_row

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate,PromptTemplate
from langchain.chains import ConversationChain, LLMChain
from langchain.schema.output_parser import StrOutputParser

from langchain.agents.agent_types import AgentType
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory,ConversationSummaryBufferMemory,ConversationSummaryMemory,ConversationEntityMemory
from langchain.schema import HumanMessage, AIMessage


from datetime import datetime
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage,message_to_dict,messages_from_dict

from langchain.callbacks import get_openai_callback
from datetime import datetime
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage,message_to_dict,messages_from_dict,get_buffer_string

from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

from langchain.agents.agent_types import AgentType
from langchain.agents import initialize_agent


In [5]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [6]:
llms = ChatOpenAI(model="gpt-3.5", temperature=0, api_key=OPENAI_API_KEY)

In [7]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')
    return result

In [105]:
from typing import List

class PostGresMemoryHistory(BaseChatMessageHistory):
    def __init__(self, db_name:str, db_user:str, db_password:str, db_host:str, db_port:str, table_name:str, chat_id:str):
        self.db_config = {
            'dbname': db_name,     
            'user': db_user,        
            'password': db_password,    
            'host': db_host,           
            'port': db_port,
            'table_name': table_name,                 
        }
        
        self.chat_id = chat_id
        
        
        try:
            self.conn = psycopg.connect(dbname=self.db_config['dbname'], user=self.db_config['user'], password=self.db_config['password'], host=self.db_config['host'], port=self.db_config['port'])
            self.cur = self.conn.cursor(row_factory=dict_row)
            self.create_table_if_not_exists()
        except psycopg.Error as e:
            print("Error connecting to PostgreSQL database:", e)
            self.conn = None
            self.cur = None
                  
    
    def create_table_if_not_exists(self):
        create_table_query = f"""CREATE TABLE IF NOT EXISTS {self.db_config['table_name']} (
            id SERIAL PRIMARY KEY,
            chat_id TEXT NOT NULL,
            message JSONB NOT NULL, 
            time_stamp TIMESTAMP NOT NULL
        );"""
        self.cur.execute(create_table_query)
        self.conn.commit()
    
    def add_ai_message(self, message: str) -> None:
        return self.add_message(AIMessage(content=message))
    
    def add_user_message(self, message: str) -> None:
        return self.add_message(HumanMessage(content=message))
    
    @property
    def messages(self) -> List[BaseMessage]:  # type: ignore
        """Retrieve the messages from PostgreSQL"""
        query = (
            f"SELECT * FROM {self.db_config['table_name']} WHERE chat_id = %s ORDER BY time_stamp;"
        )
        self.cur.execute(query, (self.chat_id,))
        items = [record["message"] for record in self.cur.fetchall()]
        messages = messages_from_dict(items)
        return messages
    
    
    def add_message(self, message:BaseMessage):
        time_stamp = datetime.now()
        query = f"INSERT INTO {self.db_config['table_name']} (chat_id, message, time_stamp) VALUES (%s, %s, %s);"
        self.cur.execute(query, (self.chat_id, json.dumps(message_to_dict(message)), time_stamp))
        self.conn.commit()
        
            
    def clear(self) -> None:
        conn = psycopg.connect(self.db_config)
        cur = conn.cursor()
        if conn:
            try:
                query = f"DELETE FROM {self.db_config['dbname']} WHERE session_id = %s;"
                cur.execute(query, (self.session_id,))
                conn.commit()
                cur.close()
                conn.close()
            except psycopg.Error as e:
                print("Error clearing DB:", e)
        else:
            print("Failed to establish connection...")

In [96]:
chat_id = '485276c7-8d9b-48b1-a8aa-ad65a7118192'

In [106]:
base_storage = PostGresMemoryHistory(db_host=os.environ.get('db_host'), db_name=os.environ.get('db_name'), db_user=os.environ.get('db_username'), db_password=os.environ.get('db_password'), db_port=os.environ.get('db_port'), table_name=os.environ.get('conversation_table_name'), chat_id=chat_id)


In [110]:
memory  = ConversationEntityMemory(chat_memory=base_storage, llm=llms, return_messages=True,verbose=True)

In [111]:
conversation_entity= ConversationChain(
    llm=llms, 
    memory=memory,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    verbose=True,
)

In [44]:
count_tokens(
    conversation_entity, 
    'How AI will extract information from the conversation?'
)



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'AI can extract information from the conversation by analyzing the text and identifying key words, phrases, and patterns. It can understand the context of the conversation and use that information to generate relevant responses. Additionally, AI can use its training data and knowledge base to provide accurate and informative answers to questions. It can also learn from previous interactions and use that knowledge to improve its responses over time. Overall, AI uses a combination of natural language processing, machine learning, and pattern recognition techniques to extract information from the conversation and generate appropriate responses.'

Spent 2240 tokens

In [45]:
count_tokens(
    conversation_entity, 
    'And what about in medical field?'
)



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'In the medical field, AI algorithms are used to extract information from medical conversations in a similar way as in other domains. AI can analyze the text and identify key medical terms, symptoms, diagnoses, and treatment options. It can also understand the context of the conversation and use that information to generate relevant responses. AI algorithms can be trained on large amounts of medical data, such as electronic health records, medical literature, and clinical guidelines, to provide accurate and up-to-date information. Additionally, AI can learn from previous medical interactions and use that knowledge to improve its responses over time. This can be particularly useful in tasks such as medical diagnosis, treatment recommendations, and patient monitoring. Overall, AI has the potential to greatly assist in the medical field by extracting valuable information from conversations and providing insights that can aid in decision-making and improve patient care.'

In [46]:
count_tokens(
    conversation_entity, 
    'Can you provide me context about the earlier conversation we had about Medical?'
)



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'Certainly! In our earlier conversation about Medical, we discussed algorithms used to detect abnormalities in analyzing medical images. Some of the commonly used algorithms mentioned were Convolutional Neural Networks (CNN), Support Vector Machines (SVM), Random Forests, Decision Trees, and Deep Learning algorithms. We also talked about how AI can extract information from conversations by analyzing the text, identifying key words and phrases, and understanding the context. In the medical field, AI algorithms can be trained on medical data to extract information such as medical terms, symptoms, diagnoses, and treatment options. AI can also learn from previous medical interactions to improve its responses over time. Overall, AI has the potential to greatly assist in the medical field by extracting valuable information from conversations and providing insights that can aid in decision-making and improve patient care. Is there anything specific you would like to know or discuss further?'

In [100]:
count_tokens(
    conversation_entity, 
    'Can you provide me context about the earlier conversation we had about algorithums?'
)

memory_variables
load_memory_variables
memory_variables
buffer
buffer


> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, y

'Certainly! In our earlier conversation about algorithms, we discussed the use of AI algorithms in the medical field. We mentioned algorithms such as Convolutional Neural Networks (CNN), Support Vector Machines (SVM), Random Forests, Decision Trees, and Deep Learning algorithms. These algorithms are used to analyze medical images and detect abnormalities. We also talked about how AI can extract information from conversations by analyzing the text, identifying key words and phrases, and understanding the context. In the medical field, AI algorithms can be trained on medical data to extract information such as medical terms, symptoms, diagnoses, and treatment options. AI can also learn from previous medical interactions to improve its responses over time. Overall, AI has the potential to greatly assist in the medical field by extracting valuable information from conversations and providing insights that can aid in decision-making and improve patient care. Is there anything specific you w

In [102]:
count_tokens(
    conversation_entity, 
    'What are top DeepLearning algorithums used in Medical field?'
)

memory_variables
load_memory_variables
memory_variables
buffer
buffer


> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, y

'In the medical field, some of the top Deep Learning algorithms used include Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), and Generative Adversarial Networks (GAN). CNNs are commonly used for medical image analysis, such as detecting abnormalities in X-rays, MRIs, and CT scans. RNNs are often used for tasks involving sequential data, such as analyzing time-series data from patient monitoring devices or electronic health records. GANs can be used for tasks such as generating synthetic medical images or augmenting existing datasets. These Deep Learning algorithms have shown great promise in various medical applications and continue to be an active area of research and development.'